# RPS - Rock Paper Scissors Agent - using PPL

In this notebook I will show an expirment of RPS game simulation.
I will use two players:
<ol>
    <li>Simple Player - playing according to Categorical seed (alpha vector)</li>
    <li> Inferencing Player - models the opponent as a Probalistic Program and by observations trying to infer the latent alpha vector.<br>
        Using this infered vector the player will try to exploit the simple player.<br>
</ol>
    

In [3]:
import numpy as np
import scipy as sp
import pymc3 as pm
import logging
logger = logging.getLogger('pymc3')
logger.setLevel(logging.FATAL)

WARNING (theano.configdefaults): g++ not available, if using conda: `conda install m2w64-toolchain`
C:\ProgramData\Anaconda3\lib\site-packages\theano\configdefaults.py:560: UserWarning: DeprecationWarning: there is no c++ compiler.This is deprecated and with Theano 0.11 a c++ compiler will be mandatory
  warnings.warn("DeprecationWarning: there is no c++ compiler."
WARNING (theano.configdefaults): g++ not detected ! Theano will be unable to execute optimized C-implementations (for both CPU and GPU) and will default to Python implementations. Performance will be severely degraded. To remove this warning, set Theano flags cxx to an empty string.
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


### Simple Player
The simple player creates a categorical distribution (with dirichlet prior) and a given alpha vector and returns <num_of_samples> samples from this distirbution

### Smart Player

Infercing Player, takes the toolset of MCMC infernce. Each round this player takes the moves of the simple players as observations, and uses the same Probabalistic Model to posterior inference

### RPS Model
Probabalistic Program, describes the decision process.<br>
    $dir \sim Dirichlet(\alpha_1, \alpha_2, \alpha_3)$<br>
    $Action \sim Categorical(dir)$
    


In [4]:
def rps_player_model(alpha=[1, 1, 1], observed=None):
    with pm.Model() as model:
        dirichlet = pm.Dirichlet('dirichlet', a=alpha)
        phi = pm.Categorical('phi', p=dirichlet, observed=observed)
        return model

In [5]:
def beats(i):
    """
    mapping between move and the the beating move.
    i.e beats(ROCK) == PAPER
    """
    return (i + 1) % 3

## The hierarchy

### Base class of all players

In [6]:
class Player:
    def __init__(self, id):
        self.id = id

    def move(self, history):
        raise NotImplementedError()

### Naive player

In [7]:
class NaivePlayer(Player):
    """Naive player chooses a move according to fixed probabilities.
    """
    def __init__(self, id, p=[1, 1, 1]):
        Player.__init__(self, id)
        p = np.array(p)
        p = p/sum(p)
        self.p = p
    
    def move(self, history):
        return np.argmax(sp.stats.multinomial.rvs(1, self.p))

### Frequentist Players

In [8]:
class FrequentistPlayer(Player):
    """Frequentist player uses prior history 
    to choose a move
    """
    def __init__(self, id, counts=None):
        Player.__init__(self, id)
        if counts is None:
            counts = [1, 1, 1]
        self.counts = counts
    
    def stats(self, history):
        counts = self.counts[:]
        for id, m in history:
            if id != self.id:
                counts[m] += 1
        return np.array(counts)

In [10]:
class FixedFrequentistPlayer(FrequentistPlayer):
    def __init__(self, id, counts=None):
        FrequentistPlayer.__init__(self, id, counts)
        
    def move(self, history):
        counts = self.stats(history)
        return beats(np.argmax(counts))
    
# Example
ffp = FixedFrequentistPlayer(1)
print([ffp.move([(1, 1), (2, 1), (1, 0), (2, 1)]) for _ in range(10)])
print([ffp.move([(1, 1), (2, 2), (1, 0), (2, 1), (1, 2), (2, 2)]) for _ in range(10)])

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [12]:
class RandomFrequentistPlayer(FrequentistPlayer):
    def __init__(self, id, counts=None):
        FrequentistPlayer.__init__(self, id, counts)
        
    def move(self, history):
        counts = self.stats(history)
        return beats(np.argmax(sp.stats.multinomial.rvs(n=1, p=counts/sum(counts))))
    
# Example
rfp = RandomFrequentistPlayer(1)
print([rfp.move([(1, 1), (2, 1), (1, 0), (2, 1)]) for _ in range(10)])
print([rfp.move([(1, 1), (2, 2), (1, 0), (2, 1), (1, 2), (2, 2)]) for _ in range(10)])

[2, 0, 2, 1, 1, 0, 2, 1, 2, 0]
[2, 1, 2, 1, 0, 0, 1, 1, 2, 0]


In [13]:
class BayesianPlayer(Player):
    def __init__(self, id, alpha=None):
        Player.__init__(self, id)
        if alpha is None:
            alpha = 1, 1, 1
        self.alpha = np.array(alpha)
        
    def opponent_model(self, history):
        pass
    
    def select_action(samples):
        pass
    
    def infer(self, model):
        with model:
            trace = pm.sample(step=pm.Metropolis(), model=model, return_inferencedata=True, progressbar=False, cores=1)
            return trace
        
    def sample_from_posterior(self, model, trace, theta_var):
        with model:
            posterior_pred = pm.sample_posterior_predictive(trace, progressbar=False)
            return beats(self.select_action(posterior_pred[theta_var]))
        
    def opponent_history(self, history):
        moves_history = []
        for idx, move in history:
            if idx != self.id:
                moves_history.append(move)
        return moves_history
    
    def move(self, history):
        history = self.opponent_history(history)
        opponent_model = self.opponent_model(history)
        trace = self.infer(opponent_model)
        return self.sample_from_posterior(opponent_model, trace, self.get_theta_var())

In [18]:
class CategoricalBaysianPlayer(BayesianPlayer):
    def __init__(self, id, alpha = None):
        BayesianPlayer.__init__(self, id, alpha)
    
    def opponent_model(self, history):
        with pm.Model() as model:
            dirichlet = pm.Dirichlet('dirichlet', a=self.alpha)
            phi = pm.Categorical('phi', p=dirichlet, observed=history)
            return model
    
    def get_theta_var(self):
        return 'phi'

In [19]:
class FixedCategoricalBaysianPlayer(CategoricalBaysianPlayer):
    def __init__(self, id, alpha = None):
        CategoricalBaysianPlayer.__init__(self, id, alpha)
    
    def select_action(self, samples):
        samples = samples.reshape(-1)
        counts = [0, 0, 0]
        # return most common sample
        for m in samples:
            counts[m] += 1
        return np.argmax(np.array(counts))
        
# Example
fcbp = FixedCategoricalBaysianPlayer(1)
print([fcbp.move([(1, 1), (2, 1), (1, 0), (2, 1)]) for _ in range(10)])
print([fcbp.move([(1, 1), (2, 2), (1, 0), (2, 1), (1, 2), (2, 2)]) for _ in range(10)])

[2, 2, 2, 2, 2, 2, 2, 2, 2, 2]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


In [20]:
class RandomCategoricalBaysianPlayer(CategoricalBaysianPlayer):
    def __init__(self, id, alpha = None):
        CategoricalBaysianPlayer.__init__(self, id, alpha)
    
    def select_action(self, samples):
        samples = samples.reshape(-1)
        counts = [0, 0, 0]
        # return most common sample
        for m in samples:
            counts[m] += 1
        counts = np.array(counts)
        return np.argmax(sp.stats.multinomial.rvs(n=1, p=counts/sum(counts)))
        
# Example
rcbp = RandomCategoricalBaysianPlayer(1)
print([rcbp.move([(1, 1), (2, 1), (1, 0), (2, 1)]) for _ in range(10)])
print([rcbp.move([(1, 1), (2, 2), (1, 0), (2, 1), (1, 2), (2, 2)]) for _ in range(10)])

[2, 0, 2, 2, 2, 2, 2, 0, 0, 2]
[2, 2, 2, 2, 2, 1, 0, 0, 1, 1]


In [21]:
ROCK = 0
PAPER = 1
SCISSORS = 2
def score(m1, m2):
    # ROCK < PAPER < SCISSORS < ROCK
    if m1==m2:
        return 0
    score = -1
    if m1 > m2:
        m1, m2 = m2, m1
        score = -score
    if m2 - m1 == 2:
        score = -score
    return score

def summerize_score(scores):
    first_wins = scores.count(1)
    ties = scores.count(0)
    second_wins = scores.count(-1)
    return first_wins, ties, second_wins

def game(player1, player2, n=100):
    # TODO play the game with two players
    history = []
    scores = []
    for i in range(n):
        m1 = player1.move(history)
        history.append([1, m1])
        m2 = player2.move(history)
        history.append([2, m2])
        scores.append(score(m1, m2))
    return scores

def play_game_and_print_summery(player1, player2):
    results = game(player1, player2)
    print(results)
    first_wins, ties, second_wins = summerize_score(results)
    print(f'player1 wins: {first_wins} ties: {ties} player2 wins: {second_wins}')

### Game between random frequentist player and naive player

In [22]:
rfp = RandomFrequentistPlayer(1)
nap = NaivePlayer(2)
play_game_and_print_summery(rfp,nap)

[-1, 0, -1, 0, -1, 1, 1, -1, 1, -1, -1, 1, 0, 0, -1, 1, 0, 0, -1, 1, 0, 1, 1, 1, 1, -1, -1, 1, -1, -1, 0, 1, 0, -1, 1, 1, -1, 0, -1, 0, 0, 1, -1, 1, 0, -1, -1, 0, 1, -1, 1, 1, -1, 0, 1, 0, 1, 0, 0, -1, -1, 0, 0, 1, 0, 0, 1, 1, -1, 1, 0, 0, 1, 0, 0, 1, 1, -1, -1, 1, -1, 1, -1, -1, 0, -1, -1, 1, 0, 0, -1, 1, -1, 0, -1, 0, -1, 0, 1, 1]
player1 wins: 34 ties: 32 player2 wins: 34


### Game between categorical bayesian player and naive player

In [23]:
rcbp = RandomCategoricalBaysianPlayer(1)
nap = NaivePlayer(2)
play_game_and_print_summery(rcbp,nap)

[1, 0, 0, 0, 1, 0, 1, -1, 1, 0, 1, 1, -1, 1, 0, -1, 0, 1, 0, 0, -1, -1, 0, -1, 1, 1, 0, -1, 1, 0, 1, -1, 0, 1, 1, 1, 0, -1, -1, 0, 0, -1, 0, 0, -1, 1, 1, -1, 0, 0, 1, 0, 0, 1, -1, -1, -1, 0, 0, 0, 0, 1, 1, 1, 1, -1, 0, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 0, -1, 1, 1, -1, -1, 1, 1, 0, 1, 1, 1, 0, 1, 1, 0, -1, 1, -1, 1, 1, 1, 0]
player1 wins: 45 ties: 32 player2 wins: 23


### Game between random frequentist player and categorical random baysian player

In [24]:
rfp = RandomFrequentistPlayer(1)
rcbp = RandomCategoricalBaysianPlayer(2)
play_game_and_print_summery(rfp,rcbp)

[1, 1, 0, 1, 0, -1, 1, 0, 0, 0, 1, 1, -1, -1, 0, 0, 0, 1, 0, -1, 0, 1, 1, 1, 0, 0, 1, 1, 0, -1, -1, 1, 1, 0, 1, 1, 1, 0, -1, 0, 0, 0, 1, 1, 1, 0, -1, 1, -1, 1, 0, 0, 0, 1, 0, -1, 1, 1, 1, 0, 0, 0, 0, 0, -1, 0, 0, 0, 0, 0, 0, 0, 0, -1, -1, 1, -1, -1, 0, 0, 0, -1, 0, -1, 1, -1, -1, -1, -1, 0, -1, 1, 1, 1, 1, 0, 0, -1, 0, -1]
player1 wins: 32 ties: 44 player2 wins: 24


### Game between random frequentist player and fixed frequentist player

In [25]:
rfp = RandomFrequentistPlayer(1)
ffp = FixedFrequentistPlayer(2)
play_game_and_print_summery(rfp,ffp)

[-1, -1, -1, -1, 0, 1, -1, -1, 0, 0, 1, -1, 1, 1, -1, -1, -1, -1, 1, -1, -1, 1, -1, -1, -1, -1, 1, -1, -1, -1, -1, -1, 0, 1, 1, 1, -1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, -1, -1, -1, 0, -1, -1, -1, -1, 0, -1, -1, -1, 0, -1, -1, -1, -1, 0, -1, 0, -1, 1, -1, 1, -1, 0, 1, -1, -1, -1, -1, -1, 1, -1, -1, 1, -1, 1, -1, 1, 1, 1, 1, 0, 1]
player1 wins: 35 ties: 11 player2 wins: 54


### Game between fixed baysien player and naive player

In [26]:
fcbp = FixedCategoricalBaysianPlayer(1)
nap = NaivePlayer(2)
play_game_and_print_summery(fcbp,nap)

[1, -1, 1, 1, 0, 0, 1, 0, 1, -1, 1, 0, 1, -1, -1, 1, 1, -1, 0, -1, -1, 0, 1, -1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 0, 0, 0, 1, 0, -1, 1, 1, -1, 0, -1, 1, 0, 1, -1, 0, 0, 1, 1, -1, 1, 0, 1, 1, 0, 1, -1, -1, 0, -1, 0, 0, -1, 0, 1, -1, 1, 1, 0, 1, 1, 0, 1, 1, -1, 1, -1, -1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 0, -1, -1, 1, -1, -1, -1, -1]
player1 wins: 42 ties: 28 player2 wins: 30


### Game between fixed baysien player and exploitable naive player

In [27]:
fcbp = FixedCategoricalBaysianPlayer(1)
nap = NaivePlayer(2, p=[5, 1, 1])
play_game_and_print_summery(fcbp,nap)

[0, 1, 1, 1, 1, 1, -1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, -1, 1, 1, 1, -1, 0, 1, 1, 0, -1, 1, 1, 1, -1, 1, 1, -1, 1, 1, -1, 1, 1, 1, -1, 1, 1, 1, 1, 1, 0, 1, -1, 1, 1, 1, 0, 1, 1, 1, 1, 1, -1, 0, 1, -1, 1, 1, 0, -1, 0, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, -1, 1, 1, 1, 1, 1, 1, 1, -1, 1, 1]
player1 wins: 73 ties: 11 player2 wins: 16


### Game between random baysien player and exploitable naive player

In [ ]:
rcbp = RandomCategoricalBaysianPlayer(1)
nap = NaivePlayer(2, p=[5, 1, 1])
play_game_and_print_summery(rcbp,nap)

### Game between fixed frequentist and exploitable naive player

In [ ]:
ffp = FixedFrequentistPlayer(1)
nap = NaivePlayer(2, p=[5, 1, 1])
play_game_and_print_summery(ffp,nap)

### Game between random frequentist and exploitable naive player

In [ ]:
rfp = RandomFrequentistPlayer(1)
nap = NaivePlayer(2, p=[5, 1, 1])
play_game_and_print_summery(rfp,nap)

### Game between fixed baysian player and random baysian player

In [ ]:
fcbp = FixedCategoricalBaysianPlayer(1)
rcbp = RandomCategoricalBaysianPlayer(2)
play_game_and_print_summery(fcbp,rcbp)

### Game between fixed frequentist and random baysian player

In [ ]:
ffp = FixedFrequentistPlayer(1)
rcbp = RandomCategoricalBaysianPlayer(2)
play_game_and_print_summery(ffp,rcbp)

### Game between random frequentist and fixed baysian player

In [ ]:
rfp = RandomFrequentistPlayer(1)
fcbp = FixedCategoricalBaysianPlayer(2)
play_game_and_print_summery(rfp,fcbp)

### Game between fixed frequentist and fixed baysian player

In [ ]:
ffp = FixedFrequentistPlayer(1)
fcbp = FixedCategoricalBaysianPlayer(2)
play_game_and_print_summery(ffp,fcbp)

### Game between random frequentist and random baysian player

In [ ]:
rfp = FixedFrequentistPlayer(1)
rcbp = FixedCategoricalBaysianPlayer(2)
play_game_and_print_summery(rfp,rcbp)

## Baysian Analysis on the results

### Summery
we can see from the expirments that the "smart" player able to exploit the simple opponent.<br> As the opponent is farther from complete random strategy we succeed to exploit it better<br> And when it plays complete random the smart player do the same and the results are even
